In [1]:
import jupman
jupman.init()


# Practical 11

In this practical we will will see some other functionalities of Biopython.

## Slides

The slides of the introduction can be found here: [Intro](docs/Practical11.pdf)

## Biopython

From the Biopython tutorial: The Biopython Project is an international association of developers of freely available Python tools for computational molecular biology. The goal of Biopython is to make it as easy as possible to use Python for bioinformatics by creating high-quality, reusable modules and classes. Biopython features include parsers for various Bioinformatics file formats (BLAST, Clustalw, FASTA, Genbank,...), access to online services (NCBI, Expasy,...), interfaces to common and not-so-common programs (Clustalw, DSSP, MSMS...), a standard sequence class, various clustering modules, a KD tree data structure etc. and even documentation.

In this practical we will see some features of Biopython but refer to [biopython documentation](http://biopython.org/wiki/Documentation) to discover all its features, recipes etc.

These notes are largely based on what available [here](http://biopython.org/DIST/docs/tutorial/Tutorial.pdf).



## BLAST

[Blast (Basic logical alignment search tool)](https://www.ncbi.nlm.nih.gov/pubmed/2231712) is a well known tool to find similarities between biological sequences. It compares DNA or protein sequences and calculates the statistical significance of the matches found.

The typical interaction with BLAST sees the user submit some sequences to the tool to get an alignment and then the hits are parsed to obtain information on the matches. Both these steps can be performed from within Biopython. Although it is possible to interact directly with a local installation of BLAST, in this practical we will work with the tool made available by NCBI (available [here](https://blast.ncbi.nlm.nih.gov/Blast.cgi)). 

### The function qblast

The online version of blast can be accessed through the ```Bio.Blast.NCBIWWW.qblast()``` function.

It's basic syntax is the following (to use it we need to make the import ```from Bio.Blast import NCBIWWW```):

```
result_handle = Bio.Blast.NCBIWWW.qblast(blast_program, database, query_str)
```
where ```blast_program``` is the program to perform the alignment. The options are **blastn, blastp, blastx, tblast or tblastx**. ```database``` is the database to search against and ```query_str``` is a string containing the query toe search against the database. The query can be a sequence or a fasta entry or an identifier like a GI number (NCIBI's sequence identification number). Among the others, some optional parameters are the output format (```format_type``` that by default is "XML" which is the most stable output format but results can be stored also as text with "Text") and ```expect``` (the e-value threshold).

Some databases to search against are reported below:

![](img/pract11/blast_dbs.png)

The query string can be obtained in different ways, for example it is possible to load sequences from a fasta file with:

```
from Bio.Blast import NCBIWWW
fasta_string = open("myfile.fasta").read()
result_handle = NCBIWWW.qblast("blastn", "nt", fasta_string)
```

or we can give a SeqRecord:

```
from Bio.Blast import NCBIWWW
from Bio import SeqIO
record = SeqIO.read("myfile.fasta", format="fasta")
result_handle = NCBIWWW.qblast("blastn", "nt", record.seq)
```

It is also possible to specify some optional paramters in the ```entrez_query``` for example we can limit the search to specific organisms with: ```entrez_query='"Malus Domestica" [Organism]'```.


### Parsing qblast output

Once the qblast call returns, it gives the results in a handle object ```result_handle``` that we can parse or we can write to disk to avoid having to rerun the query other times. If we expect to get one alignment only, we can use the method **read** otherwise (if we have multiple query sequences) we should use the method **parse**: 

```
blast_record = NCBIXML.read(result_handle)
```

or 

```
blast_records = NCBIXML.parse(result_handle)
```
Note that to use these methods we first need to import the ```NCBIXML``` module with ```from Bio.Blast import NCBIXML```.

These methods are analogous to what seen in the case of SeqIo and AlignIO. In the case of multiple entries we can loop through them with:

```
blast_records = NCBIXML.parse(result_handle)
for record in blast_records:
    #do something with it...
    
```
or we can retrieve one record at a time with ```record = next(blast_records)```.


### Saving results to file

To save the results present in the result_handle we can simply write them to file. In case we have only one entry we can read it and write it to file:

```
out_f = open("my_blast_result.xml", "w")
out_f.write(result_handle.read())
out_f.close 
result_handle.close()
```
If we have more than one entry we need to loop thorugh all the entries and save them in the file:

```
out_f = open("my_blast_result.xml", "w")
for entry in result_handle.parse():
    out_f.write(entry)
out_f.close 
result_handle.close()
```

**Example:**

Let's BLAST the galactosidase alpha (gi number: 2717) against the human database on NCBI and save the results to file. (**Note that it can take several seconds/minutes to run!**).

In [8]:
from Bio.Blast import NCBIWWW

result_handle = NCBIWWW.qblast("blastn", "nt", "2717")


with open("file_samples/blast_res.xml","w") as out_f:
    out_f.write(result_handle.read())

result_handle.close()

### Open a blast .XML file

A BLAST output file can be read by opening the file to get the handler and then parse it with the method **parse** seen above:

```
from Bio.Blast import NCBIXML
result_handle = open("my_blast.xml")
blast_records = NCBIXML.parse(result_handle)
```
This will end up in a handler to the blast results.


### The BLAST record class

The ```Bio.Blast.Record.Blast``` class holds the results of the alignment. In particular it is composed of the following three information:

1. *Descriptions* : a list of Description objects. Each ```Description``` holds the following information:

    - ```Description.title``` : a string with the title of the hit;
    - ```Description.score``` : a float with the score of the alignment;
    - ```Description.num_alignments``` : an int with the number of alignments with the same subject;
    - ```Description.e``` : a float with the e-value of the alignment.
    
2. *Alignments* : a list of Alignment objects. Each ```Alignment``` holds the following information:

    - ```Alignment.title``` : a string with the title of the hit (identical to ```Description.title```);
    - ```Alignment.length``` : an int with the length of the alignment;
    - ```Alignment.hsps``` : a list of HSP objects (High Scoring Pair). Each ```HSP``` has the following info:
        
        - ```HSP.score``` : the BLAST score of the hit
        - ```HSP.bits``` :  the bits score of the hit (x: on average 2^x pairs to find such a good hit by chance)
        - ```HSP.expect``` : the evalue of the hit
        - ```HSP.num_alignments``` : the number of alignments for the same subject
        - ```HSP.identities``` : the numbe of identities between query and subject
        - ```HSP.positives``` : the number of identical bases/aminos or having similar chemical properties
        - ```HSP.gaps``` : the number of gaps between query and subject
        - ```HSP.strand``` : a **tuple** with (query,subject) strands
        - ```HSP.frame``` : a **tuple** with the frame shifts
        - ```HSP.query/HSP.sbjct``` : query/subject sequence
        - ```HSP.query_start/HSP.sbjct_start``` :query/subject start point
        - ```HSP.match``` : the match sequence (basically "|" for matches and spaces for mismatches)
        - ```HSP.align_length``` : the alignment length.

More information on the BLAST record can be found [here](http://biopython.org/DIST/docs/api/Bio.Blast.Record-module.html).

**Example:**

Let's blast the serum albumin sequence (gi number [23307792](https://www.ncbi.nlm.nih.gov/nuccore/AF542069.1)) on the human genome and report all the information reported by BLAST. (warning might take a while to run!)

In [54]:
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML

result_handle = NCBIWWW.qblast("blastn", "nr", "23307792", 
                               entrez_query='"Homo Sapiens" [Organism]'
                               )



for res in NCBIXML.parse(result_handle):
    for d in res.descriptions:
        
        print("TITLE:{}\nSCORE:{}\nN.ALIGN:{}\nE-VAL:{}".format(
            d.title,d.score, d.num_alignments,d.e))
        
    for a in res.alignments:
        print("Align Title:{}\nAlign Len: {}".format(a.title, a.length))

        
    
        for h in a.hsps:
            s = h.score
            b = h.bits
            e = h.expect
            n = h.num_alignments
            i = h.identities
            p = h.positives
            g = h.gaps
            st = h.strand
            f = h.frame
            q = h.query
            sb = h.sbjct
            qs = h.query_start
            ss = h.sbjct_start
            qe = h.query_end
            se = h.sbjct_end
            m = h.match 
            al = h.align_length
            
            print("Score: {} Bits: {} E-val: {}".format(s,b,e))
            print("N.aligns:{} Ident:{} Pos.:{} Gaps:{} Align len:{}".format(
                n,i,p,g,al))
            print("Strand: {} Frame: {}".format(st,f))
            print("Query:", q, " start:", qs, " end:", qe)
            print("Match:",m)
            print("Subjc:",sb, " start:", ss, " end:", se)
            

result_handle.close()

TITLE:gi|23307792|gb|AF542069.1| Homo sapiens serum albumin (HSA) mRNA, complete cds
SCORE:4352.0
N.ALIGN:1
E-VAL:0.0
TITLE:gi|1046552723|ref|NM_000477.6| Homo sapiens albumin (ALB), mRNA
SCORE:4304.0
N.ALIGN:1
E-VAL:0.0
TITLE:gi|28591|emb|V00495.1| H.sapiens mRNA for serum albumin
SCORE:4252.0
N.ALIGN:2
E-VAL:0.0
TITLE:gi|7770116|gb|AF119840.1|AF119840 Homo sapiens PRO0903 mRNA, complete cds
SCORE:4062.0
N.ALIGN:1
E-VAL:0.0
TITLE:gi|25058738|gb|BC039235.1| Homo sapiens albumin, mRNA (cDNA clone IMAGE:4768004), containing frame-shift errors
SCORE:4056.0
N.ALIGN:1
E-VAL:0.0
TITLE:gi|23243417|gb|BC036003.1| Homo sapiens albumin, mRNA (cDNA clone MGC:32850 IMAGE:4724105), complete cds
SCORE:4052.0
N.ALIGN:1
E-VAL:0.0
TITLE:gi|158258946|dbj|AK292755.1| Homo sapiens cDNA FLJ78413 complete cds, highly similar to Homo sapiens albumin, mRNA
SCORE:4036.0
N.ALIGN:1
E-VAL:0.0
TITLE:gi|28589|emb|V00494.1| Human messenger RNA for serum albumin (HSA)
SCORE:4032.0
N.ALIGN:1
E-VAL:0.0
TITLE:gi|2170645

Match: ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| |||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
Subjc: GTTTTTGTATGAATATGCAAGAAGGCATCCTGATTACTCTGTCGTGCTGCTGCTGAGACTTGCCAAGACATATGAAACCACTCTAGAGAAGTGCTGTGCCGCTGCAGATCCTCATGAATGCTATGCCAAAGTG  start: 12731  end: 12863
Score: 256.0 Bits: 232.118 E-val: 1.99393e-57
N.aligns: None Idents: 131 Positives: 131 Gaps: 0 Align len: 133
Strand: (None, None) Frame: (1, 1)
Query: ATACTTATATGAAATTGCCAGAAGACATCCTTACTTCTATGCCCCGGAACTCCTTTTCTTTGCTAAAAGGTATAAAGCTGCTTTTACAGAATGTTGCCAAGCTGCTGATAAGGCTGCCTGCCTGTTGCCAAAG  start: 516  end: 648
Match: |||||||||||||||||||||||||||||||||||| |||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| |||||||||||||||||||||
Subjc: ATACTTATATGAAATTGCCAGAAGACATCCTTACTTTTATGCCCCGGAACTCCTTTTCTTTGCTAAAAGGTATAAAGCTGCTTTTACAGAATGTTGCCAAGCTGCTGATAAAGCTGCCTGCCTGTTGCCAAAG  start: 7051  end: 7183
Score: 220.0 Bits: 199.657 E-val: 1.17852e-47
N.aligns: None Idents: 115 Posi

## Getting data from NCBI

Biopython provides a module (```Bio.Entrez```) to pull data off resources like PubMed or GenBank, and other repositories programmatically through [Entrez](http://www.ncbi.nlm.nih.gov/Entrez/). 

There are some limitations (mostly taken care directly by Biopython) that you should be aware of when you use NCBI's services. Check [here](http://www.ncbi.nlm.nih.gov/books/NBK25497/#chapter2.Usage_Guidelines_and_Requiremen) if you want to learn what these limitations are.

First of all we need to import the Entrez module with (```from Bio import Entrez```) and then we can start interacting with Entrez, then we should specify (optional) an email setting ```Entrez.email```.

In particular the module (complete info on [Entrez module are here](http://biopython.org/DIST/docs/api/Bio.Entrez-module.html)) provides, among the others, the following functions:

1. ```res_handle = Entrez.einfo(db)``` returns a summary of the Entez databases as a results handle. ```db``` is an optional paramter specifying the resource of interest;
2. ```res_handle = Entrez.esearch(db, term,id)``` returns all the entries in ```db``` having query matching the term ```term```. It is also possible to specify an ```id``` to get the information relative to that resource id;
3. ```res_handle = Entrez.efetch(db, id, rettype, retmode)``` returns full record corresponding to the identifier ``id`` from the database ``db`` formatted in ```rettype``` (eg. gb, fasta,... [complete list](https://www.ncbi.nlm.nih.gov/books/NBK25499/table/chapter4.T._valid_values_of__retmode_and/?report=objectonly)) and return mode ```retmode``` (eg. text); 
4. ```res_handle = Entrez.esummary(db, id)``` returns the summary of the entry ```id``` from the database ```db``` as a handle;
5. ```result = Entrez.read(res_handle)``` reads the information on the XML handle ```res_handle``` and stores them in a dictionary, list or string, depending on the case.  


**Example:**

Let's get a list of all available databases in Entrez as a dictionary. Let's then get a summary of the entries in 'sra'.

In [72]:
from Bio import Entrez

Entrez.email = "my_email"
handle = Entrez.einfo()
res = Entrez.read(handle)
print(res)
print("")
print("As a list:")
print(res['DbList'])

res = Entrez.read(Entrez.einfo(db = "sra"))
#uncomment to see all the information captured
#print(res)
#for el in res["DbInfo"].keys():
#    print(el) 
print("")
print("Entries count:", res["DbInfo"]["Count"])
print("LastUpdate:", res["DbInfo"]["LastUpdate"])
print("Description:", res["DbInfo"]["Description"])




{'DbList': ['pubmed', 'protein', 'nuccore', 'ipg', 'nucleotide', 'nucgss', 'nucest', 'structure', 'sparcle', 'genome', 'annotinfo', 'assembly', 'bioproject', 'biosample', 'blastdbinfo', 'books', 'cdd', 'clinvar', 'clone', 'gap', 'gapplus', 'grasp', 'dbvar', 'gene', 'gds', 'geoprofiles', 'homologene', 'medgen', 'mesh', 'ncbisearch', 'nlmcatalog', 'omim', 'orgtrack', 'pmc', 'popset', 'probe', 'proteinclusters', 'pcassay', 'biosystems', 'pccompound', 'pcsubstance', 'pubmedhealth', 'seqannot', 'snp', 'sra', 'taxonomy', 'biocollections', 'unigene', 'gencoll', 'gtr']}

As a list:
['pubmed', 'protein', 'nuccore', 'ipg', 'nucleotide', 'nucgss', 'nucest', 'structure', 'sparcle', 'genome', 'annotinfo', 'assembly', 'bioproject', 'biosample', 'blastdbinfo', 'books', 'cdd', 'clinvar', 'clone', 'gap', 'gapplus', 'grasp', 'dbvar', 'gene', 'gds', 'geoprofiles', 'homologene', 'medgen', 'mesh', 'ncbisearch', 'nlmcatalog', 'omim', 'orgtrack', 'pmc', 'popset', 'probe', 'proteinclusters', 'pcassay', 'biosy

Note that effectively **einfo** returned a handler to an object that can be read by the **read** function that produces a dictionary. This dictionary had one key only "DbList" that is the list of available databases in the first case, while the key when db was specified is "DbInfo".

**Example:**
Fetch the first three entries (retmax = 3) in pubmed that are related to the species "Malus Domestica" and report the title of the publication.

In [106]:
from Bio import Entrez

Entrez.email = "my_email"
handle = Entrez.esearch(db="pubmed", term="Malus Domestica", retmax = 3)
res = Entrez.read(handle)
for el in res.keys():
    print(el , " : ", res[el])

print("")
for ids in res["IdList"]:    
    print("Results for id:", ids)
    handle = Entrez.esummary(db="pubmed",  id = ids)
    res = Entrez.read(handle)
    #uncomment to see all info
    #print(res)
    for r in res:
        print(r["Title"])
        print(r["Source"])
        print("")


Count  :  4727
RetMax  :  3
IdList  :  ['29078754', '29073205', '29063961']
TranslationSet  :  [{'To': '"malus"[MeSH Terms] OR "malus"[All Fields] OR ("malus"[All Fields] AND "domestica"[All Fields]) OR "malus domestica"[All Fields]', 'From': 'Malus Domestica'}]
QueryTranslation  :  "malus"[MeSH Terms] OR "malus"[All Fields] OR ("malus"[All Fields] AND "domestica"[All Fields]) OR "malus domestica"[All Fields]
RetStart  :  0
TranslationStack  :  [{'Term': '"malus"[MeSH Terms]', 'Field': 'MeSH Terms', 'Count': '4086', 'Explode': 'Y'}, {'Term': '"malus"[All Fields]', 'Field': 'All Fields', 'Count': '4734', 'Explode': 'N'}, 'OR', {'Term': '"malus"[All Fields]', 'Field': 'All Fields', 'Count': '4734', 'Explode': 'N'}, {'Term': '"domestica"[All Fields]', 'Field': 'All Fields', 'Count': '5211', 'Explode': 'N'}, 'AND', 'GROUP', 'OR', {'Term': '"malus domestica"[All Fields]', 'Field': 'All Fields', 'Count': '630', 'Explode': 'N'}, 'OR', 'GROUP']

Results for id: 29078754
Comprehensive analysis 

**Example:**
Retrieve genbank formatted information of the Malus x domestica MYB domain class transcription factor (MYB1) mRNA complete cds (nucleotide database id:HM122614.1). Parse it as a SeqRecord, printing only the sequence (remember previous practical's SeqIO).

In [114]:
from Bio import Entrez
from Bio import SeqIO

Entrez.email = "my_email"
handle = Entrez.efetch(db="nucleotide", id = "HM122614.1", rettype = "gb", retmode="text")
my_seq = SeqIO.read(handle, format = "genbank")
print(handle.read())
print(my_seq)
print("")
print("SEQUENCE:")
print(my_seq.seq)


ID: HM122614.1
Name: HM122614
Description: Malus x domestica MYB domain class transcription factor (MYB1) mRNA, complete cds
Number of features: 3
/source=Malus domestica (apple)
/data_file_division=HTC
/organism=Malus domestica
/sequence_version=1
/references=[Reference(title='Transcription Factors in Apple', ...), Reference(title='The FruiTFul database; full length cDNAs of apple transcription factors', ...), Reference(title='Direct Submission', ...)]
/accessions=['HM122614']
/keywords=['HTC']
/date=15-AUG-2010
/taxonomy=['Eukaryota', 'Viridiplantae', 'Streptophyta', 'Embryophyta', 'Tracheophyta', 'Spermatophyta', 'Magnoliophyta', 'eudicotyledons', 'Gunneridae', 'Pentapetalae', 'rosids', 'fabids', 'Rosales', 'Rosaceae', 'Maloideae', 'Maleae', 'Malus']
/topology=linear
/molecule_type=mRNA
Seq('TTTGGTCTGCTGGGTAGGTACTCATAAAAACAAACCAACCGAAGCCTCCGAACC...AAA', IUPACAmbiguousDNA())

SEQUENCE:
TTTGGTCTGCTGGGTAGGTACTCATAAAAACAAACCAACCGAAGCCTCCGAACCGACCACCAATGACGGCCCCAAACGGCGCCGTCCCCAAACAAGCC

## Getting data from ExPASy

Similarly to what done with Entrez, it is possible to pull data out of ExPASy (https://www.expasy.org/) through the Bio.ExPASy module. We will not cover this in detail. All information can be found here: [Bio.ExPASy module](http://biopython.org/DIST/docs/api/toc-Bio.ExPASy-module.html).

As an example, we will see how to download a couple of SwissProt entries (the human and mouse P53 protein).

In [118]:
from Bio import ExPASy
from Bio import SwissProt

accessions = ["P04637", "P02340"] #the ids of the human and mouse proteins

for accession in accessions:
    handle = ExPASy.get_sprot_raw(accession)
    record = SwissProt.read(handle)
    print(record.entry_name)
    print(",".join(record.accessions))
    print(record.keywords)
    print(repr(record.organism))
    print(record.sequence[:30] + "...")



P53_HUMAN
P04637,Q15086,Q15087,Q15088,Q16535,Q16807,Q16808,Q16809,Q16810,Q16811,Q16848,Q2XN98,Q3LRW1,Q3LRW2,Q3LRW3,Q3LRW4,Q3LRW5,Q86UG1,Q8J016,Q99659,Q9BTM4,Q9HAQ8,Q9NP68,Q9NPJ2,Q9NZD0,Q9UBI2,Q9UQ61
['3D-structure', 'Acetylation', 'Activator', 'Alternative promoter usage', 'Alternative splicing', 'Apoptosis', 'Biological rhythms', 'Cell cycle', 'Complete proteome', 'Cytoplasm', 'Disease mutation', 'DNA-binding', 'Endoplasmic reticulum', 'Glycoprotein', 'Host-virus interaction', 'Isopeptide bond', 'Li-Fraumeni syndrome', 'Metal-binding', 'Methylation', 'Mitochondrion', 'Necrosis', 'Nucleus', 'Phosphoprotein', 'Polymorphism', 'Reference proteome', 'Repressor', 'Transcription', 'Transcription regulation', 'Tumor suppressor', 'Ubl conjugation', 'Zinc']
'Homo sapiens (Human).'
MEEPQSDPSVEPPLSQETFSDLWKLLPENN...
P53_MOUSE
P02340,Q9QUP3
['3D-structure', 'Acetylation', 'Activator', 'Apoptosis', 'Biological rhythms', 'Cell cycle', 'Complete proteome', 'Cytoplasm', 'Disease mutation', 'DNA-bindin

You can find [here](http://biopython.org/DIST/docs/api/toc-Bio.SeqRecord-module.html) all the details on how to deal with the SwissProt records.

## 3D structure and PDB

## Other things 

### Phylogenetics
### Clustering analysis

**Example:**

Load the seed alignment of the [Piwi (PF02171) family](http://pfam.xfam.org/family/Piwi) stored in the pfam (stockholm) format [PF02171_seed.sth](file_samples/PF02171_seed.sth). For each record print the start point "   " (three spaces), the first 20 bases of the sequence, "...", the final 20 bases of the sequence, "   " and the end point followed by the description and the external dbxref (if any). 

## Exercises


1. Write a python function that aligns the first five contigs in []() XXX against the NCBI nr database limiting the hits to the Malus Domestica organism (parameter entrez_query='"Malus Domestica" [Organism]' in qblast)and prints to screen the following info for each hsp: 
    1. The title;
    2. Score and e-value;
    3. The number of alignments on the same subject, the number of identities and positives and the alignment length;
    4. The number of mismatches and the list of their positions (hint: can use the match string and look for " ").
   

    
<div class="tggle" onclick="toggleVisibility('ex1');">Show/Hide Solution</div>
<div id="ex1" style="display:none;">

In [19]:
from Bio import SeqIO

record = SeqIO.read("file_samples/NC_005816.gb", "genbank")

for f in record.features:
    if(f.type == "CDS"):
        print(f.type, f.location)


CDS [86:1109](+)
CDS [1105:1888](+)
CDS [2924:3119](+)
CDS [3485:3857](+)
CDS [4342:4780](+)
CDS [4814:5888](-)
CDS [6004:6421](+)
CDS [6663:7602](+)
CDS [7788:8088](-)
CDS [8087:8360](-)


</div>

2. Write a python program that loads a pfam file (stockholm format .sth) and reports for each record of the alignment:
    1. the description of the entry
    2. the start and end points
    3. the number of gaps and the % of gaps on the total length of the alignment
    4. any external database references (dbxrefs), comma separated (

Print these information to the screen. Finally, write this information in a tab separated file (.tsv) having the following format:
```#Description\tstart\tend\tnum_gaps\tpercentage_gaps\tdbxrefs```.